In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from tqdm.auto import tqdm
tqdm.pandas()
import re

import os
import glob
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

using this [dataset](https://www.kaggle.com/datasets/nazmuddhohaansary/hadithbd-quran-full) which was scrapped from this [website](https://www.hadithbd.com/) and the code used for scrapping can be found [here](https://github.com/mnansary/hadith-srcapper)

In [2]:
csv_files = glob.glob(os.path.join('../input/hadithbd-quran-full/hadithbd/*', "*.csv"))
csvs = sorted(csv_files)

In [3]:
len(csvs)

114

In [4]:
csvs.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))


# Merge all dataframes

In [5]:
expandable_df = pd.read_csv(csvs[0])
for i in range(1,len(csvs)):
    expandable_df = pd.concat([expandable_df, pd.read_csv(csvs[i])],ignore_index=True)


In [6]:
expandable_df.translation[3]

'##:<source>আল-বায়ান<source><translation>বিচার দিবসের মালিক। <translation>\n##:<source>তাইসিরুল<source><translation>যিনি প্রতিফল দিবসের মালিক। <translation>\n##:<source>মুজিবুর রহমান<source><translation>যিনি বিচার দিনের মালিক। <translation>\n##:<source>Sahih International<source><translation>Sovereign of the Day of Recompense. <translation>\n'

# verses processing

In [7]:

expandable_df['ayat'] = expandable_df['text'].apply(lambda x: re.findall(r'[০-৯]+', x))


int(expandable_df.ayat[0][0])
for i in range(len(expandable_df.ayat)):
    surah_no = int(expandable_df.ayat[i][0])
    ayat_no = int(expandable_df.ayat[i][1])
    expandable_df.ayat[i] = f"surah {surah_no}, ayat  {ayat_no}"
expandable_df.head(2)

,filepath,text,tafsir-zakariya,tafsir-bayan,translation,ayat
0,/home/ansary/WORK/Research/hadithbd/1/001001.mp3,১:১ بِسۡمِ اللّٰهِ الرَّحۡمٰنِ الرَّحِیۡمِ ﴿۱﴾...,"<header>:১. রহমান, রহীম(১) আল্লাহর নামে।(২)<he...",<header>:(১) অনন্ত করুণাময় পরম দয়ালু আল্লাহর ন...,##:<source>আল-বায়ান<source><translation>পরম ক...,"surah 1, ayat 1"
1,/home/ansary/WORK/Research/hadithbd/1/001002.mp3,১:২ اَلۡحَمۡدُ لِلّٰهِ رَبِّ الۡعٰلَمِیۡنَ ۙ﴿۲...,"<header>:২. সকল হামদ’(১) আল্লাহর(২), যিনি সৃষ্...",<header>:(২) সমস্ত প্রশংসা[1] সারা জাহানের প্র...,##:<source>আল-বায়ান<source><translation>সমস্ত...,"surah 1, ayat 2"


# organizing translation columns seperately

In [8]:

srcs=["আল-বায়ান","তাইসিরুল","মুজিবুর রহমান","Sahih International"]
def get_xlation(x,src):
    try:
        x=x.split(f"<source>{src}<source>")[1].split("<translation>")[1]
        return x
    except Exception as e:
        return None

for src in srcs: 
    expandable_df[src]=expandable_df.translation.progress_apply(lambda x: get_xlation(x,src))
    
expandable_df = expandable_df.rename(columns={"আল-বায়ান": "আল_বায়ান", "মুজিবুর রহমান": "মুজিবুর_রহমান","Sahih International":"Sahih_International"})

expandable_df.head(2)

  0%|          | 0/6236 [00:00<?, ?it/s]

  0%|          | 0/6236 [00:00<?, ?it/s]

  0%|          | 0/6236 [00:00<?, ?it/s]

  0%|          | 0/6236 [00:00<?, ?it/s]

,filepath,text,tafsir-zakariya,tafsir-bayan,translation,ayat,আল_বায়ান,তাইসিরুল,মুজিবুর_রহমান,Sahih_International
0,/home/ansary/WORK/Research/hadithbd/1/001001.mp3,১:১ بِسۡمِ اللّٰهِ الرَّحۡمٰنِ الرَّحِیۡمِ ﴿۱﴾...,"<header>:১. রহমান, রহীম(১) আল্লাহর নামে।(২)<he...",<header>:(১) অনন্ত করুণাময় পরম দয়ালু আল্লাহর ন...,##:<source>আল-বায়ান<source><translation>পরম ক...,"surah 1, ayat 1",পরম করুণাময় অতি দয়ালু আল্লাহর নামে।,(আরম্ভ করছি) পরম করুণাময় অসীম দয়াময় আল্লাহর না...,"পরম করুণাময়, অসীম দয়ালু আল্লাহর নামে (শুরু করছ...","In the name of Allah, the Entirely Merciful, t..."
1,/home/ansary/WORK/Research/hadithbd/1/001002.mp3,১:২ اَلۡحَمۡدُ لِلّٰهِ رَبِّ الۡعٰلَمِیۡنَ ۙ﴿۲...,"<header>:২. সকল হামদ’(১) আল্লাহর(২), যিনি সৃষ্...",<header>:(২) সমস্ত প্রশংসা[1] সারা জাহানের প্র...,##:<source>আল-বায়ান<source><translation>সমস্ত...,"surah 1, ayat 2","সমস্ত প্রশংসা আল্লাহর জন্য, যিনি সৃষ্টিকুলের রব।",যাবতীয় প্রশংসা জগৎসমূহের প্রতিপালক আল্লাহরই জন...,"আল্লাহরই জন্য সমস্ত প্রশংসা, যিনি বিশ্বজগতের র...","[All] praise is [due] to Allah, Lord of the wo..."


In [9]:

# list(expandable_df.translation[0])

In [10]:
len(expandable_df.translation)

6236

# Issues


"< text >" tag contains nothing (the code cell below shows just few samples but there are a lot of just missing tafsir's observed in the dataset,bug in the scrapper?)

In [11]:
indexes = [5002,5369,4658,2593,5770,5761,5279,2241,6158,5932]
print(expandable_df['tafsir-bayan'][indexes[0]])
print(expandable_df['tafsir-bayan'][indexes[3]])
print(expandable_df['tafsir-bayan'][indexes[5]])
print(expandable_df['tafsir-bayan'][indexes[8]])
print(expandable_df['tafsir-bayan'][indexes[9]])
print("\n tafsir zakariya issues ..... \n")
print(expandable_df['tafsir-zakariya'][indexes[0]])
print(expandable_df['tafsir-zakariya'][indexes[1]])
print(expandable_df['tafsir-zakariya'][indexes[2]])
print(expandable_df['tafsir-zakariya'][indexes[4]])
print(expandable_df['tafsir-zakariya'][indexes[6]])
print(expandable_df['tafsir-zakariya'][indexes[8]])
print(expandable_df['tafsir-zakariya'][indexes[9]])



<header>:(২৪) তাদের কর্মের পুরস্কার স্বরূপ।<header>
<text>:
-
তাফসীরে আহসানুল বায়ান<text>
<header>:(১১১) আমি জানি না, হয়তো এটা তোমাদের জন্য এক পরীক্ষা এবং কিছু কালের জন্য জীবনোপভোগ।’<header>
<text>:
-
তাফসীরে আহসানুল বায়ান<text>
<header>:৪। অথবা উপদেশ গ্রহণ করত, ফলে তা তার উপকারে আসত।<header>
<text>:
-
তাফসীরে আহসানুল বায়ান<text>
<header>:২। ঠক্‌ঠক্‌কারী (মহাপ্রলয়) কি?<header>
<text>:
-
তাফসীরে আহসানুল বায়ান<text>
<header>:২। কিসে তোমাকে জানাল, রাত্রিতে যা আবির্ভূত হয় তা কি?<header>
<text>:
-
তাফসীরে আহসানুল বায়ান<text>

 tafsir zakariya issues ..... 

<header>:২৪. তাদের কাজের পুরস্কারস্বরূপ।<header>
<text>:
-
তাফসীরে জাকারিয়া<text>
<header>:৪৭. অতঃপর তোমাদের মধ্যে এমন কেউই নেই, যে তাঁকে রক্ষা করতে পারে।<header>
<text>:
-
তাফসীরে জাকারিয়া<text>
<header>:২৯. আমার কাছে কথা রদবদল হয় না এবং আমি বান্দাদের প্রতি যুলুমকারীও নই।<header>
<text>:
-
তাফসীরে জাকারিয়া<text>
<header>:১৩. এটা আছে মর্যাদা সম্পন্ন লিপিসমূহে<header>
<text>:
-
তাফসীরে জাকারিয়া<text>
<header>:৯. তারা কামনা করে য

# Re-organizing tafsir columns

In [12]:

def tafsir_processor(tafsir,tag):
    try:
        text="".join(re.findall(f'{tag}([^"]*){tag}', tafsir))

        text = re.sub('\n', '', text)
        text = re.sub('\xa0', '', text)
        text = re.sub(':', '', text)
        text = re.sub('\u200f', '', text)
        text = re.sub('\u200e', '', text)
        text = re.sub('\u200c', '', text)
        processed_tafsir = re.sub('\x0c', '', text)
    
        return processed_tafsir
    except Exception as e:
        return None


expandable_df["tafsir_bayan_headers"]=expandable_df["tafsir-bayan"].progress_apply(lambda tafsir: tafsir_processor(tafsir,'<header>'))
expandable_df["tafsir_bayan_text"]=expandable_df["tafsir-bayan"].progress_apply(lambda tafsir: tafsir_processor(tafsir,'<text>'))

expandable_df["tafsir_zakariya_headers"]=expandable_df["tafsir-zakariya"].progress_apply(lambda tafsir: tafsir_processor(tafsir,'<header>'))
expandable_df["tafsir_zakariya_text"]=expandable_df["tafsir-zakariya"].progress_apply(lambda tafsir: tafsir_processor(tafsir,'<text>'))

expandable_df.head(2)


  0%|          | 0/6236 [00:00<?, ?it/s]

  0%|          | 0/6236 [00:00<?, ?it/s]

  0%|          | 0/6236 [00:00<?, ?it/s]

  0%|          | 0/6236 [00:00<?, ?it/s]

,filepath,text,tafsir-zakariya,tafsir-bayan,translation,ayat,আল_বায়ান,তাইসিরুল,মুজিবুর_রহমান,Sahih_International,tafsir_bayan_headers,tafsir_bayan_text,tafsir_zakariya_headers,tafsir_zakariya_text
0,/home/ansary/WORK/Research/hadithbd/1/001001.mp3,১:১ بِسۡمِ اللّٰهِ الرَّحۡمٰنِ الرَّحِیۡمِ ﴿۱﴾...,"<header>:১. রহমান, রহীম(১) আল্লাহর নামে।(২)<he...",<header>:(১) অনন্ত করুণাময় পরম দয়ালু আল্লাহর ন...,##:<source>আল-বায়ান<source><translation>পরম ক...,"surah 1, ayat 1",পরম করুণাময় অতি দয়ালু আল্লাহর নামে।,(আরম্ভ করছি) পরম করুণাময় অসীম দয়াময় আল্লাহর না...,"পরম করুণাময়, অসীম দয়ালু আল্লাহর নামে (শুরু করছ...","In the name of Allah, the Entirely Merciful, t...",(১) অনন্ত করুণাময় পরম দয়ালু আল্লাহর নামে (আরম্...,‘বিসমিল্লাহ’র পূর্বে ‘আক্বরাউ’ ‘আবদাউ’ অথবা ‘আ...,"১. রহমান, রহীম(১) আল্লাহর নামে।(২)","১. সাধারণত আয়াতের অনুবাদে বলা হয়ে থাকে, পরম ..."
1,/home/ansary/WORK/Research/hadithbd/1/001002.mp3,১:২ اَلۡحَمۡدُ لِلّٰهِ رَبِّ الۡعٰلَمِیۡنَ ۙ﴿۲...,"<header>:২. সকল হামদ’(১) আল্লাহর(২), যিনি সৃষ্...",<header>:(২) সমস্ত প্রশংসা[1] সারা জাহানের প্র...,##:<source>আল-বায়ান<source><translation>সমস্ত...,"surah 1, ayat 2","সমস্ত প্রশংসা আল্লাহর জন্য, যিনি সৃষ্টিকুলের রব।",যাবতীয় প্রশংসা জগৎসমূহের প্রতিপালক আল্লাহরই জন...,"আল্লাহরই জন্য সমস্ত প্রশংসা, যিনি বিশ্বজগতের র...","[All] praise is [due] to Allah, Lord of the wo...",(২) সমস্ত প্রশংসা[1] সারা জাহানের প্রতিপালক আল...,"[1] الحَمد এর মধ্যে যে ال রয়েছে, তা استغراق (...","২. সকল হামদ’(১) আল্লাহর(২), যিনি সৃষ্টিকুলের(৩...",১. আরবী ভাষায় হামদ অর্থ নির্মল ও সম্ভ্রমপূর্ণ...


# Removing non arabic(bangla verses reference) from arabic text column

In [13]:
def arabic_text_cleaner(ar_text):
    cleaned_ar = re.findall(r'[^০-৯]+', ar_text)[1]
    return cleaned_ar
expandable_df["text"]=expandable_df.text.progress_apply(lambda ar_text: arabic_text_cleaner(ar_text))

expandable_df.head(2)


  0%|          | 0/6236 [00:00<?, ?it/s]

,filepath,text,tafsir-zakariya,tafsir-bayan,translation,ayat,আল_বায়ান,তাইসিরুল,মুজিবুর_রহমান,Sahih_International,tafsir_bayan_headers,tafsir_bayan_text,tafsir_zakariya_headers,tafsir_zakariya_text
0,/home/ansary/WORK/Research/hadithbd/1/001001.mp3,بِسۡمِ اللّٰهِ الرَّحۡمٰنِ الرَّحِیۡمِ ﴿۱﴾\nب...,"<header>:১. রহমান, রহীম(১) আল্লাহর নামে।(২)<he...",<header>:(১) অনন্ত করুণাময় পরম দয়ালু আল্লাহর ন...,##:<source>আল-বায়ান<source><translation>পরম ক...,"surah 1, ayat 1",পরম করুণাময় অতি দয়ালু আল্লাহর নামে।,(আরম্ভ করছি) পরম করুণাময় অসীম দয়াময় আল্লাহর না...,"পরম করুণাময়, অসীম দয়ালু আল্লাহর নামে (শুরু করছ...","In the name of Allah, the Entirely Merciful, t...",(১) অনন্ত করুণাময় পরম দয়ালু আল্লাহর নামে (আরম্...,‘বিসমিল্লাহ’র পূর্বে ‘আক্বরাউ’ ‘আবদাউ’ অথবা ‘আ...,"১. রহমান, রহীম(১) আল্লাহর নামে।(২)","১. সাধারণত আয়াতের অনুবাদে বলা হয়ে থাকে, পরম ..."
1,/home/ansary/WORK/Research/hadithbd/1/001002.mp3,اَلۡحَمۡدُ لِلّٰهِ رَبِّ الۡعٰلَمِیۡنَ ۙ﴿۲﴾\n...,"<header>:২. সকল হামদ’(১) আল্লাহর(২), যিনি সৃষ্...",<header>:(২) সমস্ত প্রশংসা[1] সারা জাহানের প্র...,##:<source>আল-বায়ান<source><translation>সমস্ত...,"surah 1, ayat 2","সমস্ত প্রশংসা আল্লাহর জন্য, যিনি সৃষ্টিকুলের রব।",যাবতীয় প্রশংসা জগৎসমূহের প্রতিপালক আল্লাহরই জন...,"আল্লাহরই জন্য সমস্ত প্রশংসা, যিনি বিশ্বজগতের র...","[All] praise is [due] to Allah, Lord of the wo...",(২) সমস্ত প্রশংসা[1] সারা জাহানের প্রতিপালক আল...,"[1] الحَمد এর মধ্যে যে ال রয়েছে, তা استغراق (...","২. সকল হামদ’(১) আল্লাহর(২), যিনি সৃষ্টিকুলের(৩...",১. আরবী ভাষায় হামদ অর্থ নির্মল ও সম্ভ্রমপূর্ণ...


In [14]:
final_columns = ['text','ayat','আল_বায়ান','তাইসিরুল','মুজিবুর_রহমান','Sahih_International','tafsir_bayan_headers','tafsir_bayan_text','tafsir_zakariya_headers','tafsir_zakariya_text']
expandable_df[final_columns].to_csv('ben_quran_with_tafsir.csv',index = False)

In [15]:
expandable_df.text[1]

' اَلۡحَمۡدُ لِلّٰهِ رَبِّ الۡعٰلَمِیۡنَ ۙ﴿۲﴾\nالحمد لله رب العلمین ۙ﴿۲﴾'

In [16]:
pd.read_csv(csvs[0]).text[1]

'১:২ اَلۡحَمۡدُ لِلّٰهِ رَبِّ الۡعٰلَمِیۡنَ ۙ﴿۲﴾\nالحمد لله رب العلمین ۙ﴿۲﴾'

# Final processed dataset is stored [here](https://www.kaggle.com/datasets/mobassir/bangla-quran-with-tafsir)